In [1]:
import numpy as np

In [2]:
a = np.zeros((20))

In [3]:
b = np.random.normal(size=(10))
idxs = np.arange(0, 10)
b, idxs

(array([-0.60517404, -0.95431942, -0.37357648,  2.066158  ,  1.00607983,
        -0.48525722,  0.36524785, -0.55143594, -0.07799583,  0.04149204]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [4]:
idxs = (idxs + 10 + 1) % 20

In [5]:
idxs

array([11, 12, 13, 14, 15, 16, 17, 18, 19,  0])

In [6]:
a[idxs] = b 

In [34]:
c = [0, 5, 10, 15]

In [45]:
np.random.uniform(low=c[:-1], high=c[1:])

array([ 3.29709859,  9.0837541 , 14.54885685])